# Advanced Town Customization: Custom Classification and Graph Manipulation

In this tutorial, we'll explore advanced techniques for manipulating town networks in simcronomicon:

1. **Custom Classification Functions**: Handle poorly tagged OpenStreetMap data
2. **Graph Manipulation**: Convert existing places into vaccination centers  
3. **Real-World Applications**: Deal with incomplete or inconsistent OSM data
4. **Dynamic Modifications**: Change place types during simulation setup

**Why This Matters**: Real-world OpenStreetMap data is often incomplete or inconsistently tagged. This tutorial shows how to adapt simcronomicon to work with imperfect data and customize networks for specific research scenarios.

In [55]:
import simcronomicon as scon
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import json
import os
import glob
from collections import Counter
import copy


## Step 1: Using the Same Location as Previous Tutorial

We'll use the same Uniklinik Aachen location to demonstrate customization techniques on familiar data.

In [56]:
point = (50.77583, 6.045277)  # Uniklinik Aachen
dist = 500  # 500m radius

## Step 2: Understanding Default Classification

Let's first examine what the default classification function detects in our area, then see how we can improve it.

In [57]:
town_params = scon.TownParameters(100, 10)  # Smaller for analysis
town_default = scon.Town.from_point(point, dist, town_params, file_prefix="uniklinik")

town_graph_path = "uniklinik.graphmlz" # Since we use the file_prefix "uniklinik"
town_config_path = "uniklinik_config.json"

print("DEFAULT CLASSIFICATION ANALYSIS")
print("=" * 50)

# Analyze place types found
place_type_counts = Counter()
healthcare_nodes = []
other_nodes = []
total_nodes_before_filtering = 0

# Count ALL nodes before any filtering
for node, data in town_default.town_graph.nodes(data=True):
    total_nodes_before_filtering += 1
    place_type = data.get('place_type', 'unknown')
    place_type_counts[place_type] += 1
    
    if place_type == 'healthcare_facility':
        healthcare_nodes.append(node)

print(f"TOTAL NODES FOUND: {total_nodes_before_filtering}")
print(f"NODES SAVED TO GRAPH: {len(town_default.town_graph.nodes)}")

print(f"\n🏥 Healthcare facilities saved: {len(healthcare_nodes)}")
print(f"Healthcare node IDs: {healthcare_nodes}")

scon.visualize_place_types_from_graphml(town_graph_path=town_graph_path, town_config_path=town_config_path)

[1/10] Initializing town object and parameters...
[2/10] Calculating EPSG code...
[3/10] Downloading OSM road network and building data...
[4/10] Processing building geometries...
[5/10] Matching building centroids to nearest road nodes...
[6/10] Classifying buildings...
[7/10] Annotating road graph with place types...
[8/10] Filtering out irrelevant nodes...
[9/10] Building town graph...
Computing shortest paths between filtered nodes...
Adding edges to final town graph...


100%|██████████| 78/78 [00:00<?, ?it/s]

[10/10] Saving a compressed graph and config_data...


Town graph successfully built and saved!
DEFAULT CLASSIFICATION ANALYSIS
TOTAL NODES FOUND: 78
NODES SAVED TO GRAPH: 78

🏥 Healthcare facilities saved: 4
Healthcare node IDs: [26, 32, 40, 53]


## Step 3: Create Custom Classification Function

Now let's create a custom classification function that's more aggressive at identifying places that are not well-tagged in OpenStreetMap by making an assumption that the unidentifiable nodes are either `accommodation` or `commercial`.

**IMPORTANT** If you are using a custom building classification function, you also need to provide the constructor, `all_place_types` or all of the place types you are classifying your buildings after in your custom function

In [58]:
def custom_classify_place(row):
    """
    Enhanced classification function that rescues 'other' nodes.
    
    Goal: Reduce the number of nodes classified as 'other' by being
    more aggressive and making reasonable assumptions.
    """
    # Convert all attributes to lowercase strings for comparison
    building = str(row.get("building", "")).lower()
    amenity = str(row.get("amenity", "")).lower()
    landuse = str(row.get("landuse", "")).lower()
    healthcare = str(row.get("healthcare", "")).lower()
    shop = str(row.get("shop", "")).lower()
    emergency = str(row.get("emergency", "")).lower()
    name = str(row.get("name", "")).lower()
    operator = str(row.get("operator", "")).lower()
    
    healthcare_keywords = {
        'hospital', 'clinic', 'doctor', 'doctors', 'pharmacy', 'medical',
        'health', 'care', 'treatment', 'therapy', 'dental', 'dentist',
        'laboratory', 'lab', 'diagnostic', 'radiology', 'physiotherapy',
        'rehabilitation', 'ambulance', 'emergency', 'urgent', 'medicine'
    }
    
    all_text = f"{building} {amenity} {healthcare} {shop} {name} {operator}".lower()
    
    if building in {'hospital', 'clinic', 'doctors_office'} or \
       healthcare in {'hospital', 'clinic', 'doctor', 'doctors', 'pharmacy', 'laboratory'} or \
       amenity in {'hospital', 'clinic', 'doctors', 'pharmacy', 'dentist'} or \
       shop in {'medical_supply', 'pharmacy'} or \
       emergency == 'yes':
        return 'healthcare_facility'
    
    if any(keyword in all_text for keyword in healthcare_keywords):
        return 'healthcare_facility'
    
    accommodation_indicators = {
        'residential', 'apartments', 'house', 'detached', 'dormitory', 'terrace',
        'allotment_house', 'bungalow', 'semidetached_house', 'hut', 'yes'
    }
    
    if building in accommodation_indicators or \
       landuse in {'residential'} or \
       'residential' in all_text or 'apartment' in all_text:
        return 'accommodation'
    
    commercial_keywords = {'shop', 'store', 'market', 'mall', 'restaurant', 'cafe', 'bar', 'retail'}
    if building in {'commercial', 'retail', 'supermarket', 'shop', 'service', 'sports_centre'} or \
       amenity in {'restaurant', 'bar', 'cafe', 'bank', 'fast_food', 'shopping'} or \
       landuse in {'commercial'} or \
       shop != '' or \
       any(keyword in all_text for keyword in commercial_keywords):
        return 'commercial'
    
    workplace_keywords = {'office', 'company', 'business', 'corporate', 'headquarters'}
    if building in {'office', 'factory', 'industrial', 'government'} or \
       amenity in {'office', 'factory', 'industry'} or \
       landuse in {'industrial', 'office'} or \
       any(keyword in all_text for keyword in workplace_keywords):
        return 'workplace'
    
    education_keywords = {'school', 'university', 'college', 'institute', 'academy', 'campus'}
    if building in {'school', 'university', 'kindergarten', 'college'} or \
       amenity in {'university', 'kindergarten', 'school', 'college'} or \
       any(keyword in all_text for keyword in education_keywords):
        return 'education'
    
    if building in {'chapel', 'church', 'temple', 'mosque', 'synagogue'} or \
       amenity in {'chapel', 'church', 'temple', 'mosque', 'synagogue'} or \
       landuse in {'religious'}:
        return 'religious'
    
    # AGGRESSIVE CLASSIFICATION TO REDUCE 'OTHER' NODES
    # If it has a building tag, make educated guesses
    if building in {'yes', 'building'} or building != '':
        # If near commercial keywords, assume commercial
        if any(word in all_text for word in ['shop', 'store', 'business', 'service']):
            return 'commercial'
        # Default assumption: buildings are likely accommodation
        return 'accommodation'
    
    # If it has amenity info but wasn't caught above
    if amenity != '' and amenity not in {'', 'nan', 'none'}:
        return 'commercial'  # Most amenities are commercial
    
    # Last resort: if it has any meaningful data, guess accommodation
    if any(field != '' for field in [name, landuse]) and \
       not any(field in ['', 'nan', 'none'] for field in [name, landuse]):
        return 'accommodation'
    
    # Only return 'other' if truly no useful data
    return 'other'

# IF YOU ARE HAVING A CUSTOM CLASSIFICATION FUNCTION, YOU ALSO NEED `all_place_types`
all_place_types = ['accommodation', 'commercial', 'religious', 'education', 'workplace', 'healthcare_facility']

town_custom = scon.Town.from_point(
    point, 
    dist, 
    town_params,
    classify_place_func=custom_classify_place, all_place_types= all_place_types,
    file_prefix="uniklinik_custom"
)

custom_town_graph_path = "uniklinik_custom.graphmlz"
custom_town_config_path = "uniklinik_custom_config.json"

[1/10] Initializing town object and parameters...
[2/10] Calculating EPSG code...
[3/10] Downloading OSM road network and building data...
[4/10] Processing building geometries...
[5/10] Matching building centroids to nearest road nodes...
[6/10] Classifying buildings...
[7/10] Annotating road graph with place types...
[8/10] Filtering out irrelevant nodes...
[9/10] Building town graph...
Computing shortest paths between filtered nodes...
Adding edges to final town graph...


100%|██████████| 138/138 [00:00<00:00, 5319.83it/s]

[10/10] Saving a compressed graph and config_data...


Town graph successfully built and saved!


## Step 4: Compare Default vs Custom Classification

Let's create a new town with our custom classification and see the differences.

In [59]:


print(f"\nCLASSIFICATION COMPARISON")
print("=" * 60)

# Analyze custom classification results
custom_place_type_counts = Counter()
total_custom_nodes = 0

for node, data in town_custom.town_graph.nodes(data=True):
    total_custom_nodes += 1
    place_type = data.get('place_type', 'unknown')
    custom_place_type_counts[place_type] += 1

print(f"IMPROVEMENT SUMMARY:")
print(f"   Default total nodes saved: {len(town_default.town_graph.nodes)}")
print(f"   Custom total nodes saved:  {total_custom_nodes}")
print(f"   Additional nodes rescued:  +{total_custom_nodes - len(town_default.town_graph.nodes)}")

# Side-by-side comparison
print(f"\n DETAILED COMPARISON:")
print("Place Type".ljust(20) + "Default".ljust(10) + "Custom".ljust(10) + "Change")
print("-" * 50)

# Get all place types from both classifications
all_saved_types = set(['accommodation', 'healthcare_facility', 'commercial', 'workplace', 'education', 'religious'])

for place_type in sorted(all_saved_types):
    default_count = sum(1 for n, d in town_default.town_graph.nodes(data=True) 
                       if d.get('place_type') == place_type)
    custom_count = custom_place_type_counts.get(place_type, 0)
    change = custom_count - default_count
    change_str = f"+{change}" if change > 0 else str(change)
    
    print(f"{place_type.ljust(20)}{str(default_count).ljust(10)}{str(custom_count).ljust(10)}{change_str}")

# Visualize the improved classification
scon.visualize_place_types_from_graphml(
    town_graph_path=custom_town_graph_path, 
    town_config_path=custom_town_config_path
)


CLASSIFICATION COMPARISON
IMPROVEMENT SUMMARY:
   Default total nodes saved: 78
   Custom total nodes saved:  138
   Additional nodes rescued:  +60

 DETAILED COMPARISON:
Place Type          Default   Custom    Change
--------------------------------------------------
accommodation       58        119       +61
commercial          2         2         0
education           12        10        -2
healthcare_facility 4         5         +1
religious           1         1         0
workplace           1         1         0


## Step 5: Manual Graph Manipulation - Adding Vaccination Centers

Sometimes we need to manually convert existing places into healthcare facilities to simulate policy interventions (e.g., schools becoming vaccination centers).

In [60]:
def add_vaccination_centers(town, conversion_strategy="schools_to_healthcare", conversion_percent=40):
    """
    Convert existing places into additional healthcare facilities.
    
    Parameters
    ----------
    town : Town
        The town object to modify
    conversion_strategy : str
        Strategy for converting places:
        - "schools_to_healthcare": Convert education facilities
        - "commercial_to_healthcare": Convert commercial places
        - "mixed": Convert a mix of both schools and commercial places
    conversion_percent : int
        Approximate percentage of eligible places to convert (default: 20%)
    """
    
    print(f"ADDING VACCINATION CENTERS: {conversion_strategy}")
    print("=" * 50)
    
    original_healthcare = len([n for n, d in town.town_graph.nodes(data=True) 
                              if d.get('place_type') == 'healthcare_facility'])
    
    # Count total nodes of each potential type
    education_nodes = [n for n, d in town.town_graph.nodes(data=True) 
                      if d.get('place_type') == 'education']
    
    commercial_nodes = [n for n, d in town.town_graph.nodes(data=True) 
                       if d.get('place_type') == 'commercial']
    
    # Determine how many total places to convert (roughly 20% of potential sites)
    if conversion_strategy == "schools_to_healthcare":
        eligible_nodes = education_nodes
        target_count = max(3, int(len(eligible_nodes) * conversion_percent / 100))
    elif conversion_strategy == "commercial_to_healthcare":
        eligible_nodes = commercial_nodes
        target_count = max(3, int(len(eligible_nodes) * conversion_percent / 100))
    elif conversion_strategy == "mixed":
        # For mixed, we'll do half education, half commercial
        eligible_nodes = education_nodes + commercial_nodes
        target_count = max(4, int(len(eligible_nodes) * conversion_percent / 100))
    else:
        print(f"Unknown strategy: {conversion_strategy}")
        return []
    
    # If we don't have enough eligible nodes, convert all of them
    if len(eligible_nodes) <= target_count:
        nodes_to_convert = eligible_nodes
    else:
        # Randomly select nodes to convert
        import random
        nodes_to_convert = random.sample(eligible_nodes, target_count)
    
    # Track conversions
    conversions = []
    
    # Process conversions based on strategy
    if conversion_strategy == "schools_to_healthcare":
        print(f"Converting {len(nodes_to_convert)} education facilities to vaccination centers")
        for node in nodes_to_convert:
            town.town_graph.nodes[node]['place_type'] = 'healthcare_facility'
            town.town_graph.nodes[node]['original_type'] = 'education'
            town.town_graph.nodes[node]['conversion_reason'] = 'school_vaccination_center'
            conversions.append((node, 'education', 'healthcare_facility'))
    
    elif conversion_strategy == "commercial_to_healthcare":
        print(f"Converting {len(nodes_to_convert)} commercial places to vaccination centers")
        for node in nodes_to_convert:
            town.town_graph.nodes[node]['place_type'] = 'healthcare_facility'
            town.town_graph.nodes[node]['original_type'] = 'commercial'
            town.town_graph.nodes[node]['conversion_reason'] = 'pharmacy_vaccination_center'
            conversions.append((node, 'commercial', 'healthcare_facility'))
    
    elif conversion_strategy == "mixed":
        # Split between schools and commercial
        education_count = min(len(education_nodes), target_count // 2)
        commercial_count = min(len(commercial_nodes), target_count - education_count)
        
        # Adjust if we don't have enough of one type
        if education_count < target_count // 2 and len(commercial_nodes) > commercial_count:
            commercial_count = min(len(commercial_nodes), target_count - education_count)
            
        mixed_nodes_to_convert = []
        
        # Add education nodes
        if education_count > 0:
            if education_count >= len(education_nodes):
                mixed_nodes_to_convert.extend([(node, 'education') for node in education_nodes])
            else:
                selected_education = random.sample(education_nodes, education_count)
                mixed_nodes_to_convert.extend([(node, 'education') for node in selected_education])
        
        # Add commercial nodes
        if commercial_count > 0:
            if commercial_count >= len(commercial_nodes):
                mixed_nodes_to_convert.extend([(node, 'commercial') for node in commercial_nodes])
            else:
                selected_commercial = random.sample(commercial_nodes, commercial_count)
                mixed_nodes_to_convert.extend([(node, 'commercial') for node in selected_commercial])
        
        print(f"Converting {len(mixed_nodes_to_convert)} places to vaccination centers " +
              f"({sum(1 for n, t in mixed_nodes_to_convert if t == 'education')} schools, " +
              f"{sum(1 for n, t in mixed_nodes_to_convert if t == 'commercial')} commercial)")
        
        # Process mixed conversions
        for node, original_type in mixed_nodes_to_convert:
            town.town_graph.nodes[node]['place_type'] = 'healthcare_facility'
            town.town_graph.nodes[node]['original_type'] = original_type
            
            if original_type == 'education':
                town.town_graph.nodes[node]['conversion_reason'] = 'school_vaccination_center'
            else:
                town.town_graph.nodes[node]['conversion_reason'] = 'pharmacy_vaccination_center'
                
            conversions.append((node, original_type, 'healthcare_facility'))
    
    new_healthcare = len([n for n, d in town.town_graph.nodes(data=True) 
                         if d.get('place_type') == 'healthcare_facility'])
    
    print(f"Conversion complete:")
    print(f"   Original healthcare facilities: {original_healthcare}")
    print(f"   New healthcare facilities: {new_healthcare}")
    print(f"   Added: +{new_healthcare - original_healthcare} vaccination centers")
    
    if conversions:
        print(f"\nConversion details:")
        for node, from_type, to_type in conversions[:5]:  # Show first 5
            reason = town.town_graph.nodes[node].get('conversion_reason', 'unknown')
            print(f"   Node {node}: {from_type} → {to_type} ({reason})")
        if len(conversions) > 5:
            print(f"   ... and {len(conversions) - 5} more")
    
    return conversions

# Test different conversion strategies
print("TESTING DIFFERENT CONVERSION STRATEGIES")
print("=" * 60)

# Strategy 1: Convert schools to vaccination centers
town_schools = copy.deepcopy(town_custom)
conversions_schools = add_vaccination_centers(town_schools, "schools_to_healthcare")

print("\n" + "="*60)

# Strategy 2: Convert some commercial places
town_commercial = copy.deepcopy(town_custom)
conversions_commercial = add_vaccination_centers(town_commercial, "commercial_to_healthcare")

print("\n" + "="*60)

# Strategy 3: Mixed approach
town_mixed = copy.deepcopy(town_custom)
conversions_mixed = add_vaccination_centers(town_mixed, "mixed")

TESTING DIFFERENT CONVERSION STRATEGIES
ADDING VACCINATION CENTERS: schools_to_healthcare
Converting 4 education facilities to vaccination centers
Conversion complete:
   Original healthcare facilities: 5
   New healthcare facilities: 9
   Added: +4 vaccination centers

Conversion details:
   Node 91: education → healthcare_facility (school_vaccination_center)
   Node 4: education → healthcare_facility (school_vaccination_center)
   Node 44: education → healthcare_facility (school_vaccination_center)
   Node 117: education → healthcare_facility (school_vaccination_center)

ADDING VACCINATION CENTERS: commercial_to_healthcare
Converting 2 commercial places to vaccination centers
Conversion complete:
   Original healthcare facilities: 5
   New healthcare facilities: 7
   Added: +2 vaccination centers

Conversion details:
   Node 81: commercial → healthcare_facility (pharmacy_vaccination_center)
   Node 129: commercial → healthcare_facility (pharmacy_vaccination_center)

ADDING VACCINATIO

Let's also save the modified towns and see what the towns with different vaccination distributation strategies look like.

In [61]:
# Save all modified towns
saved_files = {}
for name, town in [
    ("schools", town_schools),
    ("commercial", town_commercial),
    ("mixed", town_mixed)
]:
    graphmlz, config = town.save_to_files(f"uniklinik_{name}", overwrite=True)
    saved_files[name] = (graphmlz, config)
    print(f"Saved {name} town to: {graphmlz}, {config}")

for name, (graphmlz, config) in saved_files.items():
    print(f"\nVisualizing {name} town:")
    scon.visualize_place_types_from_graphml(
        town_graph_path=graphmlz,
        town_config_path=config
    )

Saved schools town to: uniklinik_schools.graphmlz, uniklinik_schools_config.json
Saved commercial town to: uniklinik_commercial.graphmlz, uniklinik_commercial_config.json
Saved mixed town to: uniklinik_mixed.graphmlz, uniklinik_mixed_config.json

Visualizing schools town:



Visualizing commercial town:



Visualizing mixed town:


## Step 6: Analyze Impact of Graph Modifications

Let's see how our modifications affect the spatial distribution of healthcare facilities and potential vaccination coverage.

In [62]:
def analyze_healthcare_distribution(town, title="Healthcare Distribution"):
    
    print(f"\n{title.upper()}")
    print("=" * 50)
    
    # Count healthcare facilities
    healthcare_nodes = [n for n, d in town.town_graph.nodes(data=True) 
                       if d.get('place_type') == 'healthcare_facility']
    
    total_nodes = len(town.town_graph.nodes)
    healthcare_ratio = len(healthcare_nodes) / total_nodes
    
    print(f"Total locations: {total_nodes}")
    print(f"Healthcare facilities: {len(healthcare_nodes)}")
    print(f"Healthcare coverage: {healthcare_ratio:.1%}")
    
    # Analyze conversion types if present
    conversion_types = Counter()
    original_healthcare = 0
    
    for node in healthcare_nodes:
        data = town.town_graph.nodes[node]
        if 'original_type' in data:
            reason = data.get('conversion_reason', 'converted')
            conversion_types[reason] += 1
        else:
            original_healthcare += 1
    
    if conversion_types:
        print(f"\nHealthcare facility breakdown:")
        print(f"   Original facilities: {original_healthcare}")
        for reason, count in conversion_types.items():
            print(f"   {reason.replace('_', ' ').title()}: {count}")
    
    # Calculate average distances between healthcare facilities
    if len(healthcare_nodes) > 1:
        distances = []
        for i, node1 in enumerate(healthcare_nodes):
            for node2 in healthcare_nodes[i+1:]:
                if town.town_graph.has_edge(node1, node2):
                    distance = town.town_graph[node1][node2]['weight']
                    distances.append(distance)
        
        if distances:
            avg_distance = np.mean(distances)
            min_distance = np.min(distances)
            print(f"\nSpatial analysis:")
            print(f"   Average distance between facilities: {avg_distance:.0f}m")
            print(f"   Minimum distance between facilities: {min_distance:.0f}m")
    
    return {
        'total_healthcare': len(healthcare_nodes),
        'coverage_ratio': healthcare_ratio,
        'conversion_breakdown': dict(conversion_types),
        'original_count': original_healthcare
    }

# Analyze all our modified towns
results = {}

results['original'] = analyze_healthcare_distribution(town_custom, "Original Custom Classification")
results['schools'] = analyze_healthcare_distribution(town_schools, "Schools as Vaccination Centers")
results['commercial'] = analyze_healthcare_distribution(town_commercial, "Commercial as Vaccination Centers") 
results['mixed'] = analyze_healthcare_distribution(town_mixed, "Mixed Conversion Strategy")

# Summary comparison
print(f"\nSTRATEGY COMPARISON SUMMARY")
print("=" * 50)
print("Strategy".ljust(25) + "Healthcare".ljust(12) + "Coverage".ljust(10) + "Added")
print("-" * 55)

for strategy, data in results.items():
    added = data['total_healthcare'] - results['original']['total_healthcare'] if strategy != 'original' else 0
    coverage_pct = f"{data['coverage_ratio']:.1%}"
    added_str = f"+{added}" if added > 0 else "0"
    
    print(f"{strategy.ljust(25)}{str(data['total_healthcare']).ljust(12)}{coverage_pct.ljust(10)}{added_str}")


ORIGINAL CUSTOM CLASSIFICATION
Total locations: 138
Healthcare facilities: 5
Healthcare coverage: 3.6%

Spatial analysis:
   Average distance between facilities: 531m
   Minimum distance between facilities: 186m

SCHOOLS AS VACCINATION CENTERS
Total locations: 138
Healthcare facilities: 9
Healthcare coverage: 6.5%

Healthcare facility breakdown:
   Original facilities: 5
   School Vaccination Center: 4

Spatial analysis:
   Average distance between facilities: 659m
   Minimum distance between facilities: 139m

COMMERCIAL AS VACCINATION CENTERS
Total locations: 138
Healthcare facilities: 7
Healthcare coverage: 5.1%

Healthcare facility breakdown:
   Original facilities: 5
   Pharmacy Vaccination Center: 2

Spatial analysis:
   Average distance between facilities: 652m
   Minimum distance between facilities: 186m

MIXED CONVERSION STRATEGY
Total locations: 138
Healthcare facilities: 9
Healthcare coverage: 6.5%

Healthcare facility breakdown:
   Original facilities: 5
   Pharmacy Vaccina

## Step 7: Test Impact on Vaccination Simulation

Let's run a quick vaccination simulation to see how our graph modifications affect vaccination coverage and queue dynamics.

In [63]:
def test_vaccination_simulation(town, strategy_name, timesteps=10, num_runs=5):
    
    print(f"\nVACCINATION SIMULATION: {strategy_name} ({num_runs} runs)")
    print("=" * 50)
    
    # Store results from all runs
    all_results = []
    
    total_population = 1000  # CHANGED: Increased from 500 to 1000
    for run in range(num_runs):
        # Create fresh town parameters for simulation
        sim_town_params = scon.TownParameters(total_population + 1, 1)  # 1001 people, 1 spreader
        
        # Update the town with new parameters (need fresh copy for each run)
        town_copy = copy.deepcopy(town)
        town_copy.town_params = sim_town_params
        
        # Step events for vaccination behavior
        step_events = [
            scon.StepEvent(
                "vaccination_drive",
                scon.FolkSEIQRDV.interact,
                scon.EventType.DISPERSE,
                5000,
                ['accommodation']
            )
        ]
        
        # Vaccination-focused parameters
        vax_params = scon.SEIQRDVModelParameters(
            max_energy=5,
            lam_cap=0.0,         # No immigration
            beta=0.0,            # No transmission (focus on vaccination)
            alpha=1.0,           # 100% want vaccination
            gamma=4,             # Standard incubation  
            delta=5,             # Standard quarantine delay
            lam=7,               # Recovery time
            rho=7,               # Time to death
            kappa=0.1,           # 10% mortality rate
            mu=0.0,              # No natural deaths
            hospital_capacity=10  # CHANGED: Increased from 3 to 10
        )
        
        # Rest of the function remains the same...
        model = scon.SEIQRDVModel(vax_params, step_events)
        simulation = scon.Simulation(town_copy, model, timesteps, seed=False)
        simulation.run(silent=True)
        
        # Analyze results
        with h5py.File(f"simulation_output.h5", "r") as h5file:
            summary = h5file["status_summary/summary"][:]
            
            # Track vaccination progress
            vaccination_progress = [(row['timestep'], row['V']) for row in summary]
            final_vaccinated = summary[-1]['V']
            
            coverage = final_vaccinated / total_population if total_population > 0 else 0
            
            # Find when 50% vaccination was achieved
            halfway_point = total_population * 0.5
            day_50_percent = None
            for timestep, vaccinated in vaccination_progress:
                if vaccinated >= halfway_point:
                    day_50_percent = timestep
                    break
            
            # Healthcare facility utilization
            healthcare_count = len([n for n, d in town_copy.town_graph.nodes(data=True) 
                                  if d.get('place_type') == 'healthcare_facility'])
            
            max_daily_capacity = healthcare_count * vax_params.hospital_capacity
            
            # Calculate average daily vaccination rate
            daily_rates = []
            if len(vaccination_progress) > 1:
                for i in range(1, len(vaccination_progress)):
                    daily_increase = vaccination_progress[i][1] - vaccination_progress[i-1][1]
                    daily_rates.append(daily_increase)
            
            avg_daily_rate = np.mean(daily_rates) if daily_rates else 0
            capacity_utilization = avg_daily_rate / max_daily_capacity if max_daily_capacity > 0 else 0
            
            # Store results for this run
            run_results = {
                'final_coverage': coverage,
                'day_50_percent': day_50_percent,
                'healthcare_facilities': healthcare_count,
                'max_capacity': max_daily_capacity,
                'avg_daily_rate': avg_daily_rate,
                'capacity_utilization': capacity_utilization,
                'final_vaccinated': final_vaccinated,
                'total_population': total_population
            }
            all_results.append(run_results)
    
    # Calculate averages across all runs
    avg_results = {}
    
    # Numerical averages
    numerical_fields = ['final_coverage', 'healthcare_facilities', 'max_capacity', 
                       'avg_daily_rate', 'capacity_utilization', 'final_vaccinated', 'total_population']
    
    for field in numerical_fields:
        values = [result[field] for result in all_results]
        avg_results[field] = np.mean(values)
        avg_results[f'{field}_std'] = np.std(values)
    
    # Special handling for day_50_percent (some runs might not reach 50%)
    day_50_values = [result['day_50_percent'] for result in all_results if result['day_50_percent'] is not None]
    if day_50_values:
        avg_results['day_50_percent'] = np.mean(day_50_values)
        avg_results['day_50_percent_std'] = np.std(day_50_values)
        avg_results['runs_reached_50_percent'] = len(day_50_values)
    else:
        avg_results['day_50_percent'] = None
        avg_results['day_50_percent_std'] = 0
        avg_results['runs_reached_50_percent'] = 0
    
    # Print averaged results
    print(f"   Final vaccination coverage: {avg_results['final_coverage']:.1%} (+/- {avg_results['final_coverage_std']:.1%})")
    print(f"   Final vaccinated: {avg_results['final_vaccinated']:.1f} (+/- {avg_results['final_vaccinated_std']:.1f}) / {avg_results['total_population']:.1f}")
    
    if avg_results['day_50_percent'] is not None:
        print(f"   50% vaccination achieved by day: {avg_results['day_50_percent']:.1f} (+/- {avg_results['day_50_percent_std']:.1f})")
        print(f"   Runs reaching 50%: {avg_results['runs_reached_50_percent']}/{num_runs}")
    else:
        print(f"   50% vaccination not achieved in {timesteps} days (0/{num_runs} runs)")
    
    print(f"   Healthcare facilities: {avg_results['healthcare_facilities']:.1f}")
    print(f"   Max daily vaccination capacity: {avg_results['max_capacity']:.1f}")
    print(f"   Average daily vaccination rate: {avg_results['avg_daily_rate']:.1f} (+/- {avg_results['avg_daily_rate_std']:.1f})")
    print(f"   Capacity utilization: {avg_results['capacity_utilization']:.1%} (+/- {avg_results['capacity_utilization_std']:.1%})")
    
    return avg_results

# Run the simulations with updated parameters
simulation_results = {}

print("RUNNING VACCINATION SIMULATIONS (5 runs each)")
print("=" * 60)

# Test original custom town
simulation_results['original'] = test_vaccination_simulation(
    copy.deepcopy(town_custom), "Original Custom Classification", 15, 5
)

# Test schools conversion
simulation_results['schools'] = test_vaccination_simulation(
    copy.deepcopy(town_schools), "Schools as Vaccination Centers", 15, 5  
)

# Test commercial conversion
simulation_results['commercial'] = test_vaccination_simulation(
    copy.deepcopy(town_commercial), "Commercial as Vaccination Centers", 15, 5
)

# Test mixed strategy
simulation_results['mixed'] = test_vaccination_simulation(
    copy.deepcopy(town_mixed), "Mixed Conversion Strategy", 15, 5
)

RUNNING VACCINATION SIMULATIONS (5 runs each)

VACCINATION SIMULATION: Original Custom Classification (5 runs)
   Final vaccination coverage: 58.4% (+/- 0.8%)
   Final vaccinated: 583.8 (+/- 7.9) / 1000.0
   50% vaccination achieved by day: 10.6 (+/- 0.5)
   Runs reaching 50%: 5/5
   Healthcare facilities: 5.0
   Max daily vaccination capacity: 50.0
   Average daily vaccination rate: 48.6 (+/- 0.7)
   Capacity utilization: 97.3% (+/- 1.3%)

VACCINATION SIMULATION: Schools as Vaccination Centers (5 runs)
   Final vaccination coverage: 69.0% (+/- 1.1%)
   Final vaccinated: 690.2 (+/- 10.8) / 1000.0
   50% vaccination achieved by day: 8.0 (+/- 0.0)
   Runs reaching 50%: 5/5
   Healthcare facilities: 9.0
   Max daily vaccination capacity: 90.0
   Average daily vaccination rate: 57.5 (+/- 0.9)
   Capacity utilization: 63.9% (+/- 1.0%)

VACCINATION SIMULATION: Commercial as Vaccination Centers (5 runs)
   Final vaccination coverage: 75.6% (+/- 1.1%)
   Final vaccinated: 755.6 (+/- 10.7) / 10

In [65]:
file_patterns = ["*.json", "*.graphml" , "*.graphmlz", "*.h5"]

removed_count = 0
for pattern in file_patterns:
    matching_files = glob.glob(pattern)
    for file_path in matching_files:
        try:
            os.remove(file_path)
            print(f"Removed: {file_path}")
            removed_count += 1
        except Exception as e:
            print(f"Failed to remove {file_path}: {e}")

Removed: uniklinik_commercial_config.json
Removed: uniklinik_config.json
Removed: uniklinik_custom_config.json
Removed: uniklinik_mixed_config.json
Removed: uniklinik_schools_config.json
Removed: uniklinik.graphmlz
Removed: uniklinik_commercial.graphmlz
Removed: uniklinik_custom.graphmlz
Removed: uniklinik_mixed.graphmlz
Removed: uniklinik_schools.graphmlz
Removed: simulation_output.h5
